# Fine-tuning Whisper-small with augmented consonants

## Preliminary setup

In [2]:
import datasets
import string
import re
import torch

In [3]:
torch.set_num_threads(1)

### Load WhisperFeatureExtractor

In [4]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

### Load WhisperTokenizer

In [5]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="english", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Combine To Create A WhisperProcessor

In [6]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="english", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Prepare Data

In [7]:
from whisper_normalizer.english import EnglishTextNormalizer
normalizer = EnglishTextNormalizer()

def prepare_dataset(batch, do_normalize_text=True):
    audio = batch["audio"]
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    input_str = normalizer(batch["text"]).strip() if do_normalize_text else batch["text"]
    batch["labels"] = processor.tokenizer(input_str).input_ids
    return batch

In [8]:
#wtimit_augmented_consonants = datasets.load_dataset('marczenko/augmented_consonants_wtimit')

In [9]:
#wtimit_augmented_consonants = datasets.load_from_disk('/scratch/s5754798/augmented_consonants_wtimit')
#timit_augmented_consonants.save_to_disk('/scratch/s5754798/augmented_consonants_wtimit')

In [10]:
#wtimit_augmented_consonants = wtimit_augmented_consonants.map(prepare_dataset, num_proc=2)

In [11]:
#wtimit_augmented_consonants = wtimit_augmented_consonants.remove_columns(['file', 'text', 'audio'])

In [12]:
# wtimit_augmented_consonants.save_to_disk('/scratch/s5754798/augmented_consonants_vectorized')

In [13]:
wtimit_augmented_consonants = datasets.load_from_disk('/scratch/s5754798/augmented_consonants_vectorized')

Loading dataset from disk:   0%|          | 0/30 [00:00<?, ?it/s]

In [14]:
wtimit_augmented_consonants

DatasetDict({
    train: Dataset({
        features: ['id', 'input_features', 'labels'],
        num_rows: 15219
    })
})

In [15]:
wtimit_vectorized = datasets.load_from_disk('/scratch/s5754798/wtimit_vectorized')

Loading dataset from disk:   0%|          | 0/84 [00:00<?, ?it/s]

In [16]:
wtimit_vectorized = wtimit_vectorized.remove_columns(['file', 'text', 'audio'])

In [17]:
merged_train_dataset = datasets.concatenate_datasets([wtimit_vectorized['train'], wtimit_augmented_consonants['train']])

In [18]:
wtimit_vectorized['train'] = merged_train_dataset

In [19]:
wtimit_vectorized

DatasetDict({
    train: Dataset({
        features: ['id', 'input_features', 'labels'],
        num_rows: 52814
    })
    test: Dataset({
        features: ['id', 'input_features', 'labels'],
        num_rows: 2258
    })
    dev: Dataset({
        features: ['id', 'input_features', 'labels'],
        num_rows: 2274
    })
})

## Training and Evaluation

### Load a Pre-Trained Checkpoint

In [20]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [21]:
from functools import partial

# model.generation_config.language = "english"
# model.generation_config.task = "transcribe"
# model.generation_config.is_multilingual = False
model.config.use_cache = False
model.config.dropout = 0.1
# model.config.attention_dropout = 0.1
model.generation_config.forced_decoder_ids = None
model.generate = partial(model.generate, language="english", task="transcribe")

### Define a Data Collator

In [22]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [23]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

In [24]:
import evaluate
metric = evaluate.load("wer")

In [25]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Define the Training Configuration

In [26]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./ft-augmented-consonants",  # change to a repo name of your choice
    per_device_train_batch_size=64,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-6,
    warmup_steps=100,
    max_steps=5000,
    # weight_decay=0.01,
    
    save_safetensors=False,
    
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=400,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [27]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=wtimit_vectorized['train'],
    eval_dataset=wtimit_vectorized['dev'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/home3/s5754798/.local/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [28]:
# processor.save_pretrained(training_args.output_dir)

### Training

In [ ]:
trainer.train(resume_from_checkpoint=True)

/home3/s5754798/.local/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
1200,0.027100,0.334142,19.594450
1300,0.026200,0.331054,19.722519
1400,0.025800,0.342744,20.426894
1500,0.022200,0.341645,20.336179
1600,0.019900,0.351868,22.721451
1700,0.015200,0.349755,20.538954
1800,0.014000,0.355553,20.971185
1900,0.016400,0.358078,21.430096


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
/home3/s5754798/.local/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils